### Loading Packages

In [3]:
library(dplyr)

current_date <- as.Date("2024-12-29")
file_to_write <- "test_file_29th.csv"

In [4]:
cbbdata::cbd_login(username = "Naki", password = "Duke@123")
data <- cbbdata::cbd_torvik_team_factors(year = 2025, no_bias = TRUE) 
print(data)

API Key set!# A tibble: 364 x 31
   team   conf  games  wins losses adj_t adj_o adj_d barthag   efg def_efg   ftr
   <chr>  <chr> <dbl> <dbl>  <dbl> <dbl> <dbl> <dbl>   <dbl> <dbl>   <dbl> <dbl>
 1 Auburn SEC      12    11      1  69.3  135.  94.4   0.984  59.6    44.1  28.2
 2 Houst~ B12      11     8      3  63    122.  85.9   0.982  52.9    40.5  29  
 3 Duke   ACC      12    10      2  66.9  119.  85.1   0.978  54.9    41.2  29.7
 4 Tenne~ SEC      12    12      0  67.3  119.  86.6   0.975  57.1    40.7  34.3
 5 Flori~ SEC      12    12      0  70.7  124.  93.6   0.961  54.4    42.9  32.6
 6 Gonza~ WCC      13     9      4  71.4  123.  93.6   0.959  55.5    46.1  31.7
 7 Iowa ~ B12      11    10      1  71.3  126.  96.7   0.953  57.6    47.8  40.3
 8 Maryl~ B10      13    11      2  71.3  117.  90.8   0.949  56.9    45.2  29.7
 9 Illin~ B10      11     8      3  72.8  116.  90.6   0.946  51.5    41.5  39.9
10 Alaba~ SEC      12    10      2  75.2  122.  95.9   0.944  54.7    45.2  

## Extract ACC and B10 Teams from list

In [ ]:
# Filter teams in ACC and B10 and extract into separate lists
acc_teams <- data |>
  filter(conf == "ACC") |>
  pull(team) # Extract the 'team' column as a list

b10_teams <- data |>
  filter(conf == "B10") |>
  pull(team) # Extract the 'team' column as a list

b12_teams <- data |>
  filter(conf == "B12") |>
  pull(team) # Extract the 'team' column as a list

espn_top25_week8 <- c(
  "Tennessee", "Auburn", "Iowa St.", "Duke", "Alabama", "Florida", 
  "Kansas", "Marquette", "Oregon", "Kentucky", "Connecticut", "Oklahoma", 
  "Texas A&M", "Gonzaga", "Houston", "Mississippi", "Cincinnati", 
  "Michigan St.", "Mississippi St.", "San Diego St.", 
  "Purdue", "UCLA", "Arkansas", "Illinois", "Baylor"
)

extra_games <- c("Colorado St.", "Georgia St.", "North Carolina Central", "Iowa St.", 
  "Richmond", "Boise St.", "Nevada", "Eastern Kentucky",  "Iona", "Texas St.", 
  "Army", "Northeastern", "Northwestern"
  )

unique_teams <- unique(c(acc_teams, b10_teams, b12_teams, espn_top25_week8, extra_games))

print(acc_teams)
print(b10_teams)
print(b12_teams)
print(espn_top25_week8)
print(unique_teams)

 [1] "Duke"           "Pittsburgh"     "Clemson"        "North Carolina"
 [5] "SMU"            "Louisville"     "Florida St."    "Notre Dame"    
 [9] "Stanford"       "Virginia"       "Wake Forest"    "Georgia Tech"  
[13] "Miami FL"       "California"     "Syracuse"       "Virginia Tech" 
[17] "Boston College"
 [1] "Maryland"     "Illinois"     "UCLA"         "Michigan"     "Oregon"      
 [6] "Michigan St." "Penn St."     "Purdue"       "Wisconsin"    "Nebraska"    
[11] "Iowa"         "Northwestern" "Ohio St."     "Indiana"      "USC"         
[16] "Rutgers"      "Washington"   "Minnesota"   
 [1] "Houston"       "Iowa St."      "Texas Tech"    "Kansas"       
 [5] "Arizona"       "Cincinnati"    "Baylor"        "West Virginia"
 [9] "BYU"           "Utah"          "Arizona St."   "TCU"          
[13] "Colorado"      "UCF"           "Kansas St."    "Oklahoma St." 
 [1] "Tennessee"       "Auburn"          "Iowa St."        "Duke"           
 [5] "Alabama"         "Florida"         "K

## Extract Relevant Factors

In [6]:
selected_data_columns <- data |> 
  dplyr::select(Team = team, 
                ADJO = adj_o, 
                ADJD = adj_d,
                EFG_pct = efg, 
                TO_Pct = tov_rate, 
                OR_Pct = oreb_rate,
                FTR_Pct = ftr, 
                ThreePointer_Pct = three_pt_pct)

print(selected_data_columns)

# A tibble: 364 x 8
   Team       ADJO  ADJD EFG_pct TO_Pct OR_Pct FTR_Pct ThreePointer_Pct
   <chr>     <dbl> <dbl>   <dbl>  <dbl>  <dbl>   <dbl>            <dbl>
 1 Auburn     135.  94.4    59.6   13     35.6    28.2             38  
 2 Houston    122.  85.9    52.9   14.5   38.4    29               41.5
 3 Duke       119.  85.1    54.9   15.5   34.4    29.7             35.4
 4 Tennessee  119.  86.6    57.1   16.6   38.8    34.3             36.5
 5 Florida    124.  93.6    54.4   15.2   42.2    32.6             34.3
 6 Gonzaga    123.  93.6    55.5   14.1   32.3    31.7             34.3
 7 Iowa St.   126.  96.7    57.6   13.4   33.6    40.3             35.1
 8 Maryland   117.  90.8    56.9   13.7   34.5    29.7             36.6
 9 Illinois   116.  90.6    51.5   16     36.4    39.9             32.3
10 Alabama    122.  95.9    54.7   17.6   36.7    43.4             31.2
# i 354 more rows


## Cleaning Season Schedule

In [7]:
all_games <- cbbdata::cbd_torvik_season_schedule(year = 2025)
all_games <- all_games |>
  mutate(Location = ifelse(neutral, "N", "H")) |>
  select(-neutral, -type, -year, -game_id) |>
  rename(Team = home,
         Opponent = away,
         )

## Finding Games for Particluar Data

In [8]:
upcoming_games <- all_games |>
  filter(date == current_date)
print(upcoming_games)

required_upcoming_games <- upcoming_games |>
  filter(Team %in% unique_teams | Opponent %in% unique_teams)
print(required_upcoming_games)

          date                   Team               Opponent Location
        <Date>                 <char>                 <char>   <char>
 1: 2024-12-29       Central Arkansas   Southwestern Christ.        H
 2: 2024-12-29                 Temple                Buffalo        H
 3: 2024-12-29              Texas St.           UT Arlington        H
 4: 2024-12-29             Washington                   NJIT        H
 5: 2024-12-29         North Carolina               Campbell        H
 6: 2024-12-29                Georgia     South Carolina St.        H
 7: 2024-12-29         William & Mary                   Navy        H
 8: 2024-12-29                   Iona                Harvard        H
 9: 2024-12-29           UC Riverside             St. Thomas        H
10: 2024-12-29          James Madison                 Midway        H
11: 2024-12-29           North Dakota                Waldorf        H
12: 2024-12-29           Kennesaw St.         Brewton-Parker        H
13: 2024-12-29      

In [9]:
# Extract stats for Team and Opponent
team_data <- required_upcoming_games |>
  left_join(selected_data_columns, by = c("Team" = "Team"))

opponent_data <- required_upcoming_games |>
  left_join(selected_data_columns, by = c("Opponent" = "Team")) |>
  rename_with(~ paste("Opp_", ., sep = ""))

# Remove unwanted columns
opponent_data <- opponent_data |>
  rename(Opponent = Opp_Opponent,
         Team = Opp_Team) |>
  select(-c("Opp_date","Opp_Location"))

# Combine both datasets
final_data <- team_data |>
  left_join(opponent_data, by = c("Team", "Opponent"))

# Print the final result
print(final_data)

          date           Team         Opponent Location      ADJO      ADJD
        <Date>         <char>           <char>   <char>     <num>     <num>
 1: 2024-12-29      Texas St.     UT Arlington        H 109.40287 104.37293
 2: 2024-12-29     Washington             NJIT        H 103.54871 100.97670
 3: 2024-12-29 North Carolina         Campbell        H 116.53939  99.45831
 4: 2024-12-29           Iona          Harvard        H  96.90683 106.35747
 5: 2024-12-29           Army             UTSA        H 104.12970 113.87939
 6: 2024-12-29   Northwestern     Northeastern        H 108.57571  92.57980
 7: 2024-12-29        Alabama South Dakota St.        H 122.48031  95.87835
 8: 2024-12-29       Ohio St.      Indiana St.        H 114.31573  97.61827
 9: 2024-12-29       Illinois      Chicago St.        H 116.12737  90.58129
10: 2024-12-29         Oregon        Weber St.        H 117.26267  95.40811
11: 2024-12-29       Penn St.             Penn        H 115.92260  96.62275
12: 2024-12-

In [10]:
#write.csv(final_data, file_to_write, row.names = FALSE)